In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import openpyxl

# Configuración del WebDriver (en este caso, Chrome)
options = webdriver.ChromeOptions()
options.headless = False  # Si deseas que el navegador se ejecute en segundo plano, pon esto en True

# Usar WebDriver Manager para instalar automáticamente el ChromeDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Ruta del archivo Excel donde se guardarán los datos
excel_path = r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\FOCO1.xlsx'

# Intentar abrir el archivo Excel si existe, de lo contrario, crear uno nuevo
try:
    wb = openpyxl.load_workbook(excel_path)
    ws = wb.active
except FileNotFoundError:
    wb = openpyxl.Workbook()
    ws = wb.active
    # Crear encabezados si el archivo es nuevo
    ws.append([
        "Nombre", "Marca", "Color 1", "SKU 1", "Precio Original", "Precio Final",
        "URL de la Imagen", "URL Página", "Fecha", "0", "3", "6", "9", "0-6", "6-12",
        "talla-unica", "chica", "grande", "15", "16", "17", "18", "19", "3-6", "9-12",
        "18-24", "24-36", "12", "20", "21", "22", "23", "24", "25", "26", "27", "36",
        "0-1", "1-2", "1-3", "1-4", "2", "2-3", "3-4", "4", "4-6", "4-10", "6-8", "6-10",
        "8", "8-10", "10", "10-12", "14", "28", "29", "30", "31", "32", "150", "frutal",
        "citrico", "33", "34", "35", "37", "38", "39", "40", "41", "One", "GRUPO CATEGORIA",
        "CATEGORIA", "MUNDO", "Descuento"
    ])

# Función para verificar si hay productos en la página
def hay_productos():
    try:
        productos = driver.find_elements(By.CSS_SELECTOR, '.vtex-search-result-3-x-galleryItem')
        return len(productos) > 0
    except Exception as e:
        print(f"Error al verificar productos: {e}")
        return False

# Función para extraer y formatear el nombre del producto desde la URL
def extraer_nombre_desde_url(url):
    # Extraer el segmento de la URL que está entre "https://www.ficcus.cl/" y "/p?skuId"
    try:
        nombre_segmento = url.split("https://www.ficcus.cl/")[1].split("/p?skuId")[0]
        # Reemplazar los guiones por espacios y capitalizar la primera letra de cada palabra
        nombre_formateado = nombre_segmento.replace("-", " ").title()
        return nombre_formateado
    except Exception as e:
        print(f"Error al extraer nombre de la URL: {e}")
        return None

# Función para extraer el SKU desde la URL
def extraer_sku_desde_url(url):
    # Extraer el SKU de la URL (parte después de "p?skuId=")
    try:
        sku = url.split("p?skuId=")[1].split("&")[0]  # Tomamos todo lo que esté después de "p?skuId=" hasta el siguiente "&"
        return sku
    except Exception as e:
        print(f"Error al extraer SKU de la URL: {e}")
        return None

# Función para encontrar la primera fila vacía en la hoja
def obtener_primera_fila_vacia(ws):
    for row in range(1, ws.max_row + 2):  # Comenzamos desde la fila 1 hasta la última fila ocupada + 1
        if ws.cell(row=row, column=1).value is None:
            return row
    return ws.max_row + 1  # Si no se encuentra ninguna fila vacía, se añade al final

# Iterar sobre las páginas 1 a 50
for page_number in range(1, 60):
    # URL de la página actual
    url = f'https://www.ficcus.cl/ni%C3%B1o?_q=ni%C3%B1o&map=ft&page={page_number}'

    # Abrir la página web
    driver.get(url)

    # Espera a que el contenedor principal de productos esté presente
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.vtex-search-result-3-x-galleryItem'))
        )
    except Exception as e:
        print(f"Error: No se encontraron productos en la página {page_number}. Finalizando proceso.")
        break  # Si no se encuentran productos o hay un error, terminamos el proceso

    # Verificar si hay productos en la página
    if not hay_productos():
        print(f"Error: No se encontraron productos en la página {page_number}. Finalizando proceso.")
        break  # Si no hay productos, terminamos el proceso

    # Scroll hacia abajo para cargar más productos (si hay scroll infinito)
    for _ in range(3):  # Ajusta el número de iteraciones dependiendo de cuántos productos necesitas cargar
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(2)  # Esperamos 2 segundos después de cada scroll para asegurarnos de que se carguen nuevos productos

    # Ahora, buscamos los productos cargados en la página
    productos = driver.find_elements(By.CSS_SELECTOR, '.vtex-search-result-3-x-galleryItem')

    # Verificar que realmente hemos encontrado productos después de hacer scroll
    if not productos:
        print(f"Error: No se encontraron productos después de hacer scroll en la página {page_number}. Finalizando proceso.")
        break  # Si no hay productos después del scroll, terminamos el proceso

    # Iterar sobre los productos encontrados y guardar los datos en el Excel
    for producto in productos:
        try:
            # Extraer la URL de la página del producto
            enlace = producto.find_element(By.TAG_NAME, 'a').get_attribute('href')

            # Extraer el nombre del producto desde la URL usando la función extraer_nombre_desde_url
            nombre = extraer_nombre_desde_url(enlace)

            # Extraer el SKU desde la URL usando la función extraer_sku_desde_url
            sku = extraer_sku_desde_url(enlace)

            # Extraer la marca (si está disponible, puedes adaptarlo según la página)
            marca = producto.find_element(By.CLASS_NAME, 'vtex-product-summary-2-x-productBrand').text.strip()

            # Extraer el color (si está disponible, podría ser necesario ajustar esto)
            color = producto.find_element(By.CLASS_NAME, 'vtex-product-summary-2-x-productBrand').text.strip()

            # Extraer el precio original (si existe)
            try:
                precio_lista = producto.find_element(By.CLASS_NAME, 'vtex-product-price-1-x-listPrice').text.strip()
            except:
                precio_lista = None

            # Extraer el precio con descuento
            precio_descuento = producto.find_element(By.CLASS_NAME, 'vtex-product-price-1-x-sellingPrice').text.strip()

            # Extraer el descuento (si existe)
            try:
                descuento = producto.find_element(By.CLASS_NAME, 'vtex-store-components-3-x-discountInsideContainer').text.strip()
            except:
                descuento = None

            # Extraer la URL de la imagen
            imagen_url = producto.find_element(By.TAG_NAME, 'img').get_attribute('src')

            # Extraer la fecha (si está disponible)
            fecha = time.strftime("%Y-%m-%d")

            # Rellenar columnas adicionales (estos valores pueden variar según la página y la disponibilidad)
            tallas = ["0", "3", "6", "9", "0-6", "6-12", "talla-unica", "chica", "grande", "15", "16", "17", "18", "19", "3-6", "9-12", "18-24", "24-36", "12", "20", "21", "22", "23", "24", "25", "26", "27", "36", "0-1", "1-2", "1-3", "1-4", "2", "2-3", "3-4", "4", "4-6", "4-10", "6-8", "6-10", "8", "8-10", "10", "10-12", "14", "28", "29", "30", "31", "32", "150", "frutal", "citrico", "33", "34", "35", "37", "38", "39", "40", "41", "One"]
            grupo_categoria = "GRUPO CATEGORIA"  # Ajusta según el sitio
            categoria = "CATEGORIA"  # Ajusta según el sitio
            mundo = "MUNDO"  # Ajusta según el sitio

            # Obtener la primera fila vacía
            fila_vacia = obtener_primera_fila_vacia(ws)

            # Guardar los detalles del producto en la hoja de Excel
            ws.append([nombre, marca, color, sku, precio_lista, precio_descuento, imagen_url, enlace, fecha] + tallas + [grupo_categoria, categoria, mundo, descuento])

            # Guardar el archivo Excel inmediatamente después de agregar un producto
            wb.save(excel_path)

        except Exception as e:
            print(f"Error al extraer datos de un producto: {e}")

# Cerrar el navegador después de completar el scraping
driver.quit()
